# 기본설정

In [ ]:
# 설치
!pip install xmltodict
!pip install load_dotenv
!pip install requests
!pip install tqdm

In [ ]:
# 라이브러리 호출
import requests
import xmltodict
import json
import os
from dotenv import load_dotenv
from tqdm import tqdm

In [ ]:
load_dotenv()

OC = os.getenv('OC')

In [ ]:
# mst 정보들
same_jo_mst = ['231757', '238041', '236053', '213977', '242921', '223449', '245983', '224303', '232611', '218017', '234021', '231661', '213811', '233995', '245343', '243993', '228085', '166366', '195415', '245971', '137358', '188534', '158177', '218015', '244995', '237543', '205685', '245285', '223459', '203981', '171102', '195440', '228887', '243699', '241185', '236195', '241801', '245381', '239485', '245657', '153871', '224547', '243849', '243921', '218013', '211459', '169762', '199748', '246567', '198221', '218055', '213551', '212307', '243853', '238047', '238759', '246281', '183899', '102534', '231601', '217291', '238827', '247271', '245379', '245245', '237883', '195351', '209662', '181386', '223077', '224669', '238855', '206734', '245989', '241249', '192584', '193381', '230783', '224575', '180744', '219403', '241921', '242993', '213225', '232855', '238227', '165539', '192490', '177160', '242873', '234295', '223491', '237401', '228669', '174491', '244377', '186157', '245979', '210145', '205795', '195869', '205996', '244087', '238063', '243045', '230683', '195116', '232959', '227825', '188376', '97990', '234909', '195544', '243669', '113181', '234131', '237861', '243817', '228655', '238039', '224961', '244037', '206085', '203812', '239995', '183706', '112289', '192519', '206564', '218917', '236001', '203531', '215909', '218301', '245009', '238935', '245297', '239261', '211455', '195055', '195398', '232587', '243705', '142443', '228259', '245307', '195117', '209730', '121947', '210726', '230729', '244961', '195051', '242945', '195056', '243113', '208453', '246835', '231457', '198547', '239293', '203247', '236223', '168005', '211727', '116110', '195047', '241237', '195331', '195378', '203305', '232185', '231627', '238229', '242713', '238053', '222505', '228111', '243153', '230549', '236231', '214437', '242363', '81706', '243057', '238119', '240077', '149559', '245849', '231367', '195529', '195997', '215653', '204695', '183569', '195137', '212227', '55761', '211527', '232647', '234707', '236819', '245351', '239241', '231721', '222899', '244183', '192309', '236771', '223451', '150692', '179005', '230157', '239617', '228131', '244779', '234277', '91137', '224533', '245367', '243675', '241169', '246295', '235283', '195322', '195497', '136944', '238861', '247269', '245315', '232643', '239979', '96889', '166367', '228147', '237105', '201148', '205997', '218873', '241043', '236955', '81895', '234409', '217403', '234333', '245981', '202628', '213449', '201184', '245371', '112303', '218717', '208382', '239245', '130810', '234325', '194499', '183597', '236201', '245015', '170996', '167959', '237137', '86731', '219033', '149917', '184935', '220965', '212233', '242929', '242923', '203259', '242007', '224927', '195894', '183520', '184205', '239271', '245235', '179033', '224655', '114744', '213855', '238109', '239269', '234671', '242095', '181623', '246637', '219039', '109707', '129332', '176245', '122694', '213837', '227321', '240031', '230525', '232881', '247249', '234281', '200682', '232879', '216107', '231503', '246853', '230431', '208381', '218907', '117325', '236251', '237359', '228087', '240097', '121956', '210257', '209699', '223983', '244711', '216183', '231795', '242949', '243041', '242881', '240055', '138747', '228115', '166688', '238801', '204777', '165852', '219135', '244275', '219239', '252049', '236111', '239259', '235203', '84504', '208541', '230497', '201190', '93181', '218135', '233211', '244919', '245289', '199798', '243063', '205846', '204560', '240065', '140592', '216173', '238257', '195501', '243065', '249059', '228121', '232893', '242999', '234911', '243059', '230805', '230513', '244969', '234675', '97987', '119410', '239303', '232199', '190607', '182009', '218865', '217995', '221707', '242989', '214441', '236321', '204775', '241159', '239469', '243115', '169467', '195052', '188047', '243379', '243965', '106831', '238793', '244367', '207953', '103946', '211885', '203671', '198486', '241157', '244133', '232869', '191150', '83896', '211551', '179505', '243061', '218487', '228169', '247231', '205930', '152098', '165523', '219353', '238243', '240069', '205742', '240351', '108467', '222423', '214319', '91501', '214473', '228749', '219067', '178983', '243177', '217299', '117324', '148466', '231373', '223431', '245305', '196333', '205811', '235545', '214193', '169502', '203249', '245265', '195068', '243563', '238655', '237135', '213857', '240025', '219563', '198564', '198226', '216103', '217981', '243939', '202626', '238867', '244035', '242953', '230353', '224107', '216179', '215763', '220373', '231391', '113964', '130656', '202774', '115683', '231369', '240033', '208601', '231755', '195536', '242983', '103863', '164562', '185019', '224583', '178989', '244943', '237891', '204767', '231491', '242941', '180465', '224671', '201143', '213527', '234017', '149922', '242361', '176090', '239491', '192435', '88893', '136693', '223615', '239477', '234569', '248315', '231331', '92106', '244277', '122986', '200173', '129262', '232671', '203835', '197503', '121952', '103947', '242937', '232897', '242823', '243099', '104326', '237439', '202765', '213779', '235555', '218051', '245393', '213783', '103962', '208452', '218005', '199779', '234339', '234825', '232191', '246917', '232081', '136692', '244701', '86411', '232841', '241529', '136014', '234299', '238915', '244039', '170988', '91382', '240571', '202910', '230223', '235999', '236145', '203170', '199122', '232253', '231377', '237433', '240071', '182175', '103228', '122701', '231537', '214535', '238097', '238919', '218661', '224695', '205801', '213809', '232231', '212141', '238805', '97989', '238057', '231823', '229311', '168725', '117424', '224385', '192412', '246157', '136917', '230677', '229829', '238893', '244993', '91383', '210258', '214403', '237131', '205255', '220877', '244739', '244801', '214551', '237517', '237515', '244267', '245349', '198474', '97050', '245031', '206142', '244033', '230421', '231375', '244921', '245001', '231495', '239273', '228735', '206470', '103869', '249977', '238061', '188538', '98243', '213519', '195078', '235587', '216131', '244627', '228083', '236951', '238751', '241541', '213803', '239893', '84010', '236817', '205700', '90415', '231701', '239301', '214161', '246123', '157319', '194522', '209645', '241935', '122990', '211799', '204768', '92068', '234455', '240245', '234271', '236113', '192018', '203669', '199499', '218325', '180327', '244609', '237421', '199083', '245917', '206394', '208699', '243671', '219361', '188381', '243147', '241139', '243073', '218903', '166363', '242019', '247189', '223575', '216151', '96895', '227407', '232201', '195072', '236229', '242979', '217243', '136666', '235839', '230161', '241573', '242947', '162682', '242933', '195667', '228817', '238681', '206701', '245281', '244931', '231863', '219093', '238099', '167953', '113196', '199494', '222549', '244933', '224923', '210268', '243667', '208446', '214129', '223079', '219131', '232139', '244975', '242931', '245851', '244119', '239493', '172201', '212189', '219139', '223461', '91380', '244703', '245987', '231425', '249619', '238833', '242939', '153866', '234349', '167446', '220883', '210368', '218143', '149822', '245227', '142442', '217691', '231397', '203169', '195354', '241317', '194762', '222441', '244253', '245975', '218113', '112382', '243665', '136675', '234413', '230255', '222489', '228071', '211635', '244965', '241807', '232271', '233381', '238235', '213521', '83895', '114769', '233407', '242919', '208425', '233829', '245653', '218155', '195716', '245003', '103861', '199773', '195091', '224917', '195200', '247139', '215905', '220857', '209867', '235571', '244927', '242891', '212091', '169302', '238253', '219107', '149564', '202316', '205233', '198482', '234719', '195470', '201137', '106754', '214237', '219091', '215923', '238835', '210363', '103864', '244855', '218153', '150369', '173112', '195345', '239311', '232387', '132928', '238789', '245737', '238845', '238757']
diff_jo_mst = ['243117', '239547', '218909', '233961', '246165', '228175', '243935', '237355', '105295', '245747', '238059', '235619', '149585', '243079', '239541', '242955', '243171', '61603', '237087', '222429', '234849', '241829', '214545', '203739', '228065', '246569', '242975', '238335', '195316', '236317', '199495', '238763', '235535', '113175', '238797', '180480', '244997', '224549', '149553', '240063', '232225', '228113', '195441', '225139', '228089', '230449', '230781', '238113', '232563', '210266', '230429', '233423', '239291', '245973', '242401']
err_mst = [('153886', 'unknown', 'need to check'), ('165498', 'unknown', 'need to check'), ('231781', 'unknown', 'need to check')]

# 데이터 불러오기
(테스트용)

In [ ]:
list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=elaw&page={1}&display=100'
response = requests.get(list_url)
json_data = xmltodict.parse(response.text)

In [ ]:
# keys: '@id', '법령일련번호', '현행연혁코드', '법령명한글', '법령명영문', '법령ID', '공포일자', '공포번호', '제개정구분명', '소관부처명', '법령구분명', '시행일자', '자법타법여부', '법령상세링크'
for law in json_data['LawSearch']['law'][1:2]:
    mst = law['법령일련번호']
    url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={mst}'
    json_data_en = xmltodict.parse(requests.get(url).text)
    url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={mst}'
    json_data_kr = xmltodict.parse(requests.get(url).text)
    # print(law['법령상세링크'], law['법령일련번호'])

In [ ]:
json_data_kr.keys()

In [ ]:
print("en", json_data_en['Law'].keys())
print("kr", json_data_kr['법령'].keys())

In [ ]:
json_data_en['Law']['InfSection'].keys(), json_data_kr['법령']['기본정보'].keys()

In [ ]:
len(json_data_en['Law']['JoSection']['Jo']), len(json_data_kr['법령']['조문']['조문단위'])

In [ ]:
# Cts: 내용
#

In [ ]:
json_data_en['Law']['JoSection'].keys()

* ```joNo```: 조문번호 (en:4자리, kr:정수) > ex. 0016 / 16
* joBrNo: 조문가지번호
25-2조: 25: 조문번호, 2: 조문가지번호
* ```joTtl```: 조문제목
* ```joCts```: 조문내용
    * en: ```\n```로 구분되어 있음
    * kr: key로 구분되어 있음
    * (1) : ①
    * 1 : 1


**조문내용 관련**
* 검증: 조문의 총 개수에 대한 비고

```python
len(json_data_en['Law']['JoSection']['Jo']), len(json_data_kr['법령']['조문']['조문단위'])
```


* 검증: 조문참고자료가 있을 경우 en에서 '\n'으로 split하고 마지막 제외로 개수세기


```python
len(json_data_en['Law']['JoSection']['Jo'][{index}]['joCts'].split('\n')) - (1 if '조문참고자료' in json_data_kr['법령']['조문']['조문단위'][{index}] else 0), sum([len(t['호'])+1 if '호' in t else 1 for t in json_data_kr['법령']['조문']['조문단위'][31]['항']])
```



In [ ]:
json_data_kr['법령']['조문']['조문단위'][31]

In [ ]:
json_data_kr['법령']['조문']['조문단위'][31]['항'], json_data_en['Law']['JoSection']['Jo'][31]['joCts'].split('\n')

In [ ]:
len(json_data_en['Law']['JoSection']['Jo'][31]['joCts'].split('\n')) - (1 if '조문참고자료' in json_data_kr['법령']['조문']['조문단위'][31] else 0), sum([len(t['호'])+1 if '호' in t else 1 for t in json_data_kr['법령']['조문']['조문단위'][31]['항']])

In [ ]:
len(json_data_en['Law']['ArSection']['Ar']), len(json_data_kr['법령']['부칙']['부칙단위'])

In [ ]:
json_data_en['Law']['JoSection']

In [ ]:
len(json_data_en['Law']['ArSection']['Ar']), json_data_kr['법령']['부칙']['부칙단위'][0]

In [ ]:
import xmltodict
import json

# 한글이 포함된 XML 데이터 예시
xml_data = '''<root>
    <person>
        <name>홍길동</name>
        <age>30</age>
    </person>
    <person>
        <name>이순신</name>
        <age>25</age>
    </person>
</root>'''

# XML을 파싱하여 JSON으로 변환
json_data = json.dumps(xmltodict.parse(xml_data), indent=4, ensure_ascii=False)

# JSON 데이터 출력
print(json_data)


# 데이터 불러오기 (코드 정리)

In [ ]:
list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=elaw&page={30}&display=100'
response = requests.get(list_url)
json_data = xmltodict.parse(response.text)

In [ ]:
response

In [ ]:
json_data['LawSearch'].keys()

## 법령 일련번호(MST) 가져오기

In [ ]:
# 데이터 개수 en: 2586 kr: 5273
# 예상페이지 수: 26page (2586 // 100)
# 예상페이지 수: 53page (5273 // 100)

# MST(일련번호) 모두 가져오기
en_mst = set()
kr_mst = set()

for page in tqdm(range(1,30)):
    # print(page, "/ 26")
    list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=elaw&page={page}&display=100'
    response = requests.get(list_url)
    json_data = xmltodict.parse(response.text)
    if 'law' not in json_data['LawSearch'] or len(json_data['LawSearch']['law']) == 0:
        break
    for law in json_data['LawSearch']['law']:
        en_mst.add(law['법령일련번호'])

for page in tqdm(range(1,60)):
    # print(page, "/ 53")
    list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=law&page={page}&display=100'
    response = requests.get(list_url)
    json_data = xmltodict.parse(response.text)
    if 'law' not in json_data['LawSearch'] or len(json_data['LawSearch']['law']) == 0:
        break
    for law in json_data['LawSearch']['law']:
        kr_mst.add(law['법령일련번호'])

In [ ]:
# 현행법령이면서 영문이 있는 것
target_mst = list(kr_mst & en_mst)
len(target_mst)

In [ ]:
rt

## 법령 Parallel 만들기 (매칭시키기)
- 데이터 개수와 조항 기준으로 확인

1. API키 사용하여 [목록] 데이터 크롤링
2. 한글, 영어 각각의 [목록] 개수 파악
3. 각각의 [MST]를 가져오고 겹치는 것들을 위주로 파악
4. 데이터를 보면서 파악
    * 예시: [법률 제 18487호] 119구조ㆍ구급에 관한 법률 (25-2조에서 형식이 다름을 파악)
5. 데이터를 단계적으로 검증이 필요함
    * 데이터를 불러오면서 발생하는 문제점은 우선 `try` `catch`로 처리하여 오류 케이스 파악
* [확인1] 법령 [조항] 크롤링할 때 [조]의 개수가 같은지를 기본적으로 파악
* [확인2] [항]/[조]의 개수 같은지 파악
    * en은 `\n`기준으로 파악


### Check 01: [조문]의 개수가 같은지
1. [조문]의 개수가 같은지 파악
    - 1) [조문]의 개수가 다르다.
        - 분석 필요
    - 2) [조문]의 개수가 같다.
        - 1)번 항목의 경우를 확인하여 더블체크


In [ ]:
# 857
from tqdm import tqdm

same_jo_mst = []
diff_jo_mst = []
err_mst = []
# err_kr_mst = []


# 법령별 정보 추출
for mst in tqdm(target_mst):
    try:
        # en
        url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={mst}'
        response_en = requests.get(url)
        if response_en.status_code != 200:
            err_mst.append((mst, response_en.status_code, 'en'))
            continue

        # kr
        url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={mst}'
        response_kr = requests.get(url)
        if response_kr.status_code != 200:
            err_mst.append((mst, response_kr.status_code, 'kr'))
            continue

        # en, kr parsing
        json_data_en = xmltodict.parse(response_en.text)
        json_data_kr = xmltodict.parse(response_kr.text)

        # check 1
        # 조문개수 확인
        if len(json_data_en['Law']['JoSection']['Jo']) == len(json_data_kr['법령']['조문']['조문단위']):
            same_jo_mst.append(mst)
        else:
            diff_jo_mst.append(mst)
    except:
        err_mst.append((mst, 'unknown','need to check'))

In [ ]:
print("same")
print(same_jo_mst)
print("diff")
print(diff_jo_mst)
print("err")
print(err_mst)

In [ ]:
json_data_en['Law']['JoSection']

In [ ]:
# [조문]의 개수가 같은 것과 다른 [MST] & Error 발생 [MST]
same_jo_mst = ['231757', '238041', '236053', '213977', '242921', '223449', '245983', '224303', '232611', '218017', '234021', '231661', '213811', '233995', '245343', '243993', '228085', '166366', '195415', '245971', '137358', '188534', '158177', '218015', '244995', '237543', '205685', '245285', '223459', '203981', '171102', '195440', '228887', '243699', '241185', '236195', '241801', '245381', '239485', '245657', '153871', '224547', '243849', '243921', '218013', '211459', '169762', '199748', '246567', '198221', '218055', '213551', '212307', '243853', '238047', '238759', '246281', '183899', '102534', '231601', '217291', '238827', '247271', '245379', '245245', '237883', '195351', '209662', '181386', '223077', '224669', '238855', '206734', '245989', '241249', '192584', '193381', '230783', '224575', '180744', '219403', '241921', '242993', '213225', '232855', '238227', '165539', '192490', '177160', '242873', '234295', '223491', '237401', '228669', '174491', '244377', '186157', '245979', '210145', '205795', '195869', '205996', '244087', '238063', '243045', '230683', '195116', '232959', '227825', '188376', '97990', '234909', '195544', '243669', '113181', '234131', '237861', '243817', '228655', '238039', '224961', '244037', '206085', '203812', '239995', '183706', '112289', '192519', '206564', '218917', '236001', '203531', '215909', '218301', '245009', '238935', '245297', '239261', '211455', '195055', '195398', '232587', '243705', '142443', '228259', '245307', '195117', '209730', '121947', '210726', '230729', '244961', '195051', '242945', '195056', '243113', '208453', '246835', '231457', '198547', '239293', '203247', '236223', '168005', '211727', '116110', '195047', '241237', '195331', '195378', '203305', '232185', '231627', '238229', '242713', '238053', '222505', '228111', '243153', '230549', '236231', '214437', '242363', '81706', '243057', '238119', '240077', '149559', '245849', '231367', '195529', '195997', '215653', '204695', '183569', '195137', '212227', '55761', '211527', '232647', '234707', '236819', '245351', '239241', '231721', '222899', '244183', '192309', '236771', '223451', '150692', '179005', '230157', '239617', '228131', '244779', '234277', '91137', '224533', '245367', '243675', '241169', '246295', '235283', '195322', '195497', '136944', '238861', '247269', '245315', '232643', '239979', '96889', '166367', '228147', '237105', '201148', '205997', '218873', '241043', '236955', '81895', '234409', '217403', '234333', '245981', '202628', '213449', '201184', '245371', '112303', '218717', '208382', '239245', '130810', '234325', '194499', '183597', '236201', '245015', '170996', '167959', '237137', '86731', '219033', '149917', '184935', '220965', '212233', '242929', '242923', '203259', '242007', '224927', '195894', '183520', '184205', '239271', '245235', '179033', '224655', '114744', '213855', '238109', '239269', '234671', '242095', '181623', '246637', '219039', '109707', '129332', '176245', '122694', '213837', '227321', '240031', '230525', '232881', '247249', '234281', '200682', '232879', '216107', '231503', '246853', '230431', '208381', '218907', '117325', '236251', '237359', '228087', '240097', '121956', '210257', '209699', '223983', '244711', '216183', '231795', '242949', '243041', '242881', '240055', '138747', '228115', '166688', '238801', '204777', '165852', '219135', '244275', '219239', '252049', '236111', '239259', '235203', '84504', '208541', '230497', '201190', '93181', '218135', '233211', '244919', '245289', '199798', '243063', '205846', '204560', '240065', '140592', '216173', '238257', '195501', '243065', '249059', '228121', '232893', '242999', '234911', '243059', '230805', '230513', '244969', '234675', '97987', '119410', '239303', '232199', '190607', '182009', '218865', '217995', '221707', '242989', '214441', '236321', '204775', '241159', '239469', '243115', '169467', '195052', '188047', '243379', '243965', '106831', '238793', '244367', '207953', '103946', '211885', '203671', '198486', '241157', '244133', '232869', '191150', '83896', '211551', '179505', '243061', '218487', '228169', '247231', '205930', '152098', '165523', '219353', '238243', '240069', '205742', '240351', '108467', '222423', '214319', '91501', '214473', '228749', '219067', '178983', '243177', '217299', '117324', '148466', '231373', '223431', '245305', '196333', '205811', '235545', '214193', '169502', '203249', '245265', '195068', '243563', '238655', '237135', '213857', '240025', '219563', '198564', '198226', '216103', '217981', '243939', '202626', '238867', '244035', '242953', '230353', '224107', '216179', '215763', '220373', '231391', '113964', '130656', '202774', '115683', '231369', '240033', '208601', '231755', '195536', '242983', '103863', '164562', '185019', '224583', '178989', '244943', '237891', '204767', '231491', '242941', '180465', '224671', '201143', '213527', '234017', '149922', '242361', '176090', '239491', '192435', '88893', '136693', '223615', '239477', '234569', '248315', '231331', '92106', '244277', '122986', '200173', '129262', '232671', '203835', '197503', '121952', '103947', '242937', '232897', '242823', '243099', '104326', '237439', '202765', '213779', '235555', '218051', '245393', '213783', '103962', '208452', '218005', '199779', '234339', '234825', '232191', '246917', '232081', '136692', '244701', '86411', '232841', '241529', '136014', '234299', '238915', '244039', '170988', '91382', '240571', '202910', '230223', '235999', '236145', '203170', '199122', '232253', '231377', '237433', '240071', '182175', '103228', '122701', '231537', '214535', '238097', '238919', '218661', '224695', '205801', '213809', '232231', '212141', '238805', '97989', '238057', '231823', '229311', '168725', '117424', '224385', '192412', '246157', '136917', '230677', '229829', '238893', '244993', '91383', '210258', '214403', '237131', '205255', '220877', '244739', '244801', '214551', '237517', '237515', '244267', '245349', '198474', '97050', '245031', '206142', '244033', '230421', '231375', '244921', '245001', '231495', '239273', '228735', '206470', '103869', '249977', '238061', '188538', '98243', '213519', '195078', '235587', '216131', '244627', '228083', '236951', '238751', '241541', '213803', '239893', '84010', '236817', '205700', '90415', '231701', '239301', '214161', '246123', '157319', '194522', '209645', '241935', '122990', '211799', '204768', '92068', '234455', '240245', '234271', '236113', '192018', '203669', '199499', '218325', '180327', '244609', '237421', '199083', '245917', '206394', '208699', '243671', '219361', '188381', '243147', '241139', '243073', '218903', '166363', '242019', '247189', '223575', '216151', '96895', '227407', '232201', '195072', '236229', '242979', '217243', '136666', '235839', '230161', '241573', '242947', '162682', '242933', '195667', '228817', '238681', '206701', '245281', '244931', '231863', '219093', '238099', '167953', '113196', '199494', '222549', '244933', '224923', '210268', '243667', '208446', '214129', '223079', '219131', '232139', '244975', '242931', '245851', '244119', '239493', '172201', '212189', '219139', '223461', '91380', '244703', '245987', '231425', '249619', '238833', '242939', '153866', '234349', '167446', '220883', '210368', '218143', '149822', '245227', '142442', '217691', '231397', '203169', '195354', '241317', '194762', '222441', '244253', '245975', '218113', '112382', '243665', '136675', '234413', '230255', '222489', '228071', '211635', '244965', '241807', '232271', '233381', '238235', '213521', '83895', '114769', '233407', '242919', '208425', '233829', '245653', '218155', '195716', '245003', '103861', '199773', '195091', '224917', '195200', '247139', '215905', '220857', '209867', '235571', '244927', '242891', '212091', '169302', '238253', '219107', '149564', '202316', '205233', '198482', '234719', '195470', '201137', '106754', '214237', '219091', '215923', '238835', '210363', '103864', '244855', '218153', '150369', '173112', '195345', '239311', '232387', '132928', '238789', '245737', '238845', '238757']
diff_jo_mst = ['243117', '239547', '218909', '233961', '246165', '228175', '243935', '237355', '105295', '245747', '238059', '235619', '149585', '243079', '239541', '242955', '243171', '61603', '237087', '222429', '234849', '241829', '214545', '203739', '228065', '246569', '242975', '238335', '195316', '236317', '199495', '238763', '235535', '113175', '238797', '180480', '244997', '224549', '149553', '240063', '232225', '228113', '195441', '225139', '228089', '230449', '230781', '238113', '232563', '210266', '230429', '233423', '239291', '245973', '242401']
err_mst = [('153886', 'unknown', 'need to check'), ('165498', 'unknown', 'need to check'), ('231781', 'unknown', 'need to check')]

### Check 02: 조문의 항들을 기준으로 처리
- 같은 항에도 en에는 여러 버전이 있는 경우가 있었음
[순서]
1. `kr`에서 조문의 항들이 유니크한지 파악
    - `set()`과 `list()`개수가 같은지
2. `kr`에서 조문의 [항]과 `en`에서 조문의 [항]이 일치하는지
    - 일치
        - 1)
    - 불일치: 일반적으로 `kr`에서의 [항] 하나에 여러 `en`의 [항]이 연결된다. 방법은 아래와 같다.
        - 1) 제거 -> 이쪽이 더 나을듯
        - 2) 매칭

In [ ]:
url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={243117}'
response_en = requests.get(url)
json_data_en = xmltodict.parse(response_en.text)

url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={243117}'
response_kr = requests.get(url)
json_data_kr = xmltodict.parse(response_kr.text)

In [ ]:
fpr
int(json_data_kr['법령']['조문']['조문단위'][0]['조문번호']),int(json_data_en['Law']['JoSection']['Jo'][0]['joNo'])

In [ ]:
tmp = list()
curr = -1
for t in json_data_kr['법령']['조문']['조문단위']:
    if t['조문여부'] == '조문':
        if '조문가지번호' in t:
            tmp.append(f"{int(curr)}-{t['조문번호']}-{t['조문가지번호']}")#({t['조문시행일자'] if '조문시행일자' in t else ''})")
        else:
            tmp.append(f"{int(curr)}-{t['조문번호']}")
    if t['조문여부'] == '전문':
        curr = t['조문번호']
tmp2 = list()
curr = -1
for t in json_data_en['Law']['JoSection']['Jo']:
    if t['joYn'] == 'Y':
        if 'joBrNo' in t and int(t['joBrNo']) != 0:
            tmp2.append(f"{int(curr)}-{int(t['joNo'])}-{int(t['joBrNo'])}")
        else:
            tmp2.append(f"{int(curr)}-{int(t['joNo'])}")
    if t['joYn'] == 'N':
        curr = t['joNo']

In [ ]:
with open('Data/kr/kr_55761.json', 'r') as f:
    json_data_kr = json.load(f)
with open('Data/en/en_55761.json', 'r') as f:
    json_data_en = json.load(f)

In [ ]:
def check_jo(json_data_kr, json_data_en):
    tmp = list()
    curr = -1
    for t in json_data_kr['법령']['조문']['조문단위']:
        if t['조문여부'] == '조문':
            if '조문가지번호' in t:
                tmp.append(f"{int(curr)}-{t['조문번호']}-{t['조문가지번호']}")#({t['조문시행일자'] if '조문시행일자' in t else ''})")
            else:
                tmp.append(f"{int(curr)}-{t['조문번호']}")
        if t['조문여부'] == '전문':
            curr = t['조문번호']
    tmp2 = list()
    curr = -1
    for t in json_data_en['Law']['JoSection']['Jo']:
        if t['joYn'] == 'Y':
            if 'joBrNo' in t and int(t['joBrNo']) != 0:
                tmp2.append(f"{int(curr)}-{int(t['joNo'])}-{int(t['joBrNo'])}")
            else:
                tmp2.append(f"{int(curr)}-{int(t['joNo'])}")
        if t['joYn'] == 'N':
            curr = t['joNo']
    return tmp, tmp2 # kr, en

using_mst = []
for file in os.listdir('Data/kr/'):
    _, a = os.path.split(f'Data/kr/{file}')
    filename, form = os.path.splitext(a)
    filename = filename[3:]
    with open(f'Data/kr/{file}', 'r') as f:
        json_data_kr = json.load(f)
    with open(f'Data/en/en_{filename}.json', 'r') as f:
        json_data_en = json.load(f)
    cnt_kr, cnt_en = check_jo(json_data_kr, json_data_en)
    if cnt_kr == cnt_en:
        using_mst.append(filename)
        print("---------------")
        print(filename)
        print(cnt_kr)
        print(cnt_en)
    # print(len(cnt_kr), len(cnt_en), len(set(cnt_kr)), len(set(cnt_en)))

In [ ]:
# using case
# Branch Jo : 241921
# No Jo : 245981
mst = 241921
with open(f'Data/kr/kr_{mst}.json', 'r') as f:
    json_data_kr = json.load(f)
with open(f'Data/en/en_{mst}.json', 'r') as f:
    json_data_en = json.load(f)


In [ ]:
len(json_data_kr['법령']['조문']['조문단위'])

In [ ]:
len(json_data_en['Law']['JoSection']['Jo'])

In [ ]:
kr

In [ ]:
for kr, en in zip(json_data_kr['법령']['조문']['조문단위'], json_data_en['Law']['JoSection']['Jo']):
    cnt = 1
    # print(kr['조문내용'])
    if '항' in kr:
        kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
        cnt += len(kr_h)
        for kr_hang in kr_h:
            if '항내용' not in kr_hang:
                cnt -= 1
            else:
                pass
                # print("\t", kr_hang['항내용'])
            if '호' in kr_hang:
                kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                cnt += len(kr_hos)
                for kr_ho in kr_hos:
                    if '호내용' not in kr_ho:
                        cnt -= 1
                    else:
                        pass
                        # print("\t\t", kr_ho['호내용'])
                    if '목' in kr_ho:
                        cnt += len(kr_ho['목'])
                        for kr_mok in kr_ho['목']:
                            if '목내용' not in kr_mok:
                                cnt -= 1
                            else:
                                # print("\t\t\t", kr_mok['목내용'])
                                pass
    en_val = len(en['joCts'].replace(') (1)', ')\n(1)').split('\n'))
    # en_val1 = ((len(en['joCts'].split('\n')) + (1 if ') (1)' in en['joCts'].split('\n')[0] else 0)))# - (0 if '항' in kr else 1)
    en_val2 = sum([1 if t.strip().startswith('[') else 0 for t in en['joCts'].split('\n')])
    en_val = en_val - en_val2
    if cnt != en_val:
        pass
        # print( '(1)' in en['joCts'],cnt, en_val, -en_val2)
        # print(kr, en)
        # print(kr['조문내용'])
        # cnt = 1
        # if '항' in kr:
        #     kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
        #     cnt += len(kr_h)
        #     for kr_hang in kr_h:
        #         if '항내용' not in kr_hang:
        #             cnt -= 1
        #         else:
        #             pass
        #             print("\t", kr_hang['항내용'])
        #         if '호' in kr_hang:
        #             kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
        #             cnt += len(kr_hos)
        #             for kr_ho in kr_hos:
        #                 if '호내용' not in kr_ho:
        #                     cnt -= 1
        #                 else:
        #                     print("\t\t", kr_ho['호내용'])
        #                     pass

        #                 if '목' in kr_ho:
        #                     cnt += len(kr_ho['목'])
        #                     for kr_mok in kr_ho['목']:
        #                         if '목내용' not in kr_mok:
        #                             cnt -= 1
        #                         else:
        #                             print("\t\t\t", kr_mok['목내용'])
        #                             pass
        #                 print(cnt)
        # for t in en['joCts'].split('\n'):
        #     print(t)
        # print()
        # print('----------------------')
        # kr_cnt = len(en['joCts'].split('\n')) - (1 if '조문참고자료' in kr else 0)
        # en_cnt = None
        # case = -1
        # if len(kr['항']) == 1:
        #     case = 1
        #     en_cnt = (sum([len(kr['항']['호'])+1 if '호' in kr['항'] else 0]))
        # else:
        #     case = 2
        #     en_cnt = (sum([len(t['호'])+1 if '호' in t else 1 for t in kr['항']]))
        # if kr_cnt != en_cnt:
        #     print(case)
        #     print(kr_cnt, kr)
        #     print(en_cnt, en['joCts'].split('\n'))
    else:
        # print(kr['조문내용'], en['joCts'])
        en_law = en['joCts'].replace(') (1)', ')\n  (1)').split('\n')
        cnt = 0
        if '항' in kr:
            kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
            print(kr['조문내용'], en_law[cnt])
            cnt += 1
            for kr_hang in kr_h:
                if '항내용' not in kr_hang:
                    pass
                else:
                    pass
                    print(kr_hang['항내용'], en_law[cnt])
                    cnt += 1
                if '호' in kr_hang:
                    kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                    for kr_ho in kr_hos:
                        if '호내용' not in kr_ho:
                            pass
                        else:
                            print("\t\t", kr_ho['호내용'], en_law[cnt])
                            cnt += 1
                            pass

                        if '목' in kr_ho:
                            for kr_mok in kr_ho['목']:
                                if '목내용' not in kr_mok:
                                    pass
                                else:
                                    print("\t\t\t", kr_mok['목내용'], en_law[cnt])
                                    cnt += 1
                                    pass

In [ ]:
en_data, kr_data = [], []
for file in os.listdir('Data/kr/'):
    _, a = os.path.split(f'Data/kr/{file}')
    filename, form = os.path.splitext(a)
    filename = filename[3:]
    with open(f'Data/kr/{file}', 'r') as f:
        json_data_kr = json.load(f)
    with open(f'Data/en/en_{filename}.json', 'r') as f:
        json_data_en = json.load(f)
    cnt_kr, cnt_en = check_jo(json_data_kr, json_data_en)
    if cnt_kr == cnt_en:
        with open(f'Data/kr/kr_{mst}.json', 'r') as f:
            json_data_kr = json.load(f)
        with open(f'Data/en/en_{mst}.json', 'r') as f:
            json_data_en = json.load(f)
        for kr, en in zip(json_data_kr['법령']['조문']['조문단위'], json_data_en['Law']['JoSection']['Jo']):
            cnt = 1
            # print(kr['조문내용'])
            if '항' in kr:
                kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
                cnt += len(kr_h)
                for kr_hang in kr_h:
                    if '항내용' not in kr_hang:
                        cnt -= 1
                    else:
                        pass
                        # print("\t", kr_hang['항내용'])
                    if '호' in kr_hang:
                        kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                        cnt += len(kr_hos)
                        for kr_ho in kr_hos:
                            if '호내용' not in kr_ho:
                                cnt -= 1
                            else:
                                pass
                                # print("\t\t", kr_ho['호내용'])
                            if '목' in kr_ho:
                                cnt += len(kr_ho['목'])
                                for kr_mok in kr_ho['목']:
                                    if '목내용' not in kr_mok:
                                        cnt -= 1
                                    else:
                                        # print("\t\t\t", kr_mok['목내용'])
                                        pass
            en_val = len(en['joCts'].replace(') (1)', ')\n(1)').split('\n'))
            en_val2 = sum([1 if t.strip().startswith('[') else 0 for t in en['joCts'].split('\n')])
            en_val -= en_val2
            if cnt == en_val:
                en_law = en['joCts'].replace(') (1)', ')\n  (1)').split('\n')
                cnt = 0
                if '항' in kr:
                    kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
                    # print(kr['조문내용'], en_law[cnt])
                    kr_data.append(kr['조문내용'])
                    en_data.append(en_law[cnt])
                    cnt += 1
                    for kr_hang in kr_h:
                        if '항내용' not in kr_hang:
                            pass
                        else:
                            pass
                            # print(kr_hang['항내용'], en_law[cnt])
                            kr_data.append(kr_hang['항내용'])
                            en_data.append(en_law[cnt])
                            cnt += 1
                        if '호' in kr_hang:
                            kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
                            for kr_ho in kr_hos:
                                if '호내용' not in kr_ho:
                                    pass
                                else:
                                    # print("\t\t", kr_ho['호내용'], en_law[cnt])
                                    kr_data.append(kr_ho['호내용'])
                                    en_data.append(en_law[cnt])
                                    cnt += 1
                                    pass

                                if '목' in kr_ho:
                                    for kr_mok in kr_ho['목']:
                                        if '목내용' not in kr_mok:
                                            pass
                                        else:
                                            # print("\t\t\t", kr_mok['목내용'], en_law[cnt])
                                            kr_data.append(kr_mok['목내용'])
                                            en_data.append(en_law[cnt])
                                            cnt += 1
                                            pass

In [ ]:
import googletrans

translator = googletrans.Translator()

In [ ]:
import random
my_list = [i for i in range(len(kr_data))]
random.shuffle(my_list)
for i in my_list[:10]:
    print(kr_data[i])
    print(translator.translate(en_data[i].strip(), dest='ko').text)
    print(en_data[i].strip())
    print()

In [ ]:
with open('kr.txt','w') as f:
    for k in kr_data:
        f.write(f"{k.strip()}\n")
with open('en.txt','w') as f:
    for k in en_data:
        f.write(f"{k.strip()}\n")

In [ ]:
len(en_data)

In [ ]:
en_law = en['joCts'].replace(') (1)', ')\n  (1)').split('\n')
cnt = 0
if '항' in kr:
    kr_h = [kr['항']] if type(kr['항']) == dict else kr['항']
    print(kr['조문내용'], en_law[cnt])
    cnt += 1
    for kr_hang in kr_h:
        if '항내용' not in kr_hang:
            pass
        else:
            pass
            print(kr_hang['항내용'], en_law[cnt])
            cnt += 1
        if '호' in kr_hang:
            kr_hos = [kr_hang['호']] if type(kr_hang['호']) == dict else kr_hang['호']
            for kr_ho in kr_hos:
                if '호내용' not in kr_ho:
                    pass
                else:
                    print("\t\t", kr_ho['호내용'], en_law[cnt])
                    cnt += 1
                    pass

                if '목' in kr_ho:
                    for kr_mok in kr_ho['목']:
                        if '목내용' not in kr_mok:
                            pass
                        else:
                            print("\t\t\t", kr_mok['목내용'], en_law[cnt])
                            cnt += 1
                            pass

In [ ]:
_, a = os.path.split('Data/kr/kr_55761.json')
filename, _ = os.path.splitext(a)
filename[3:], _

In [ ]:
json_data_kr['법령']['조문']

In [ ]:
tmp = list()
curr = -1
for t in json_data_kr['법령']['조문']['조문단위']:
    if t['조문여부'] == '조문':
        if '조문가지번호' in t:
            tmp.append(f"{int(curr)}-{t['조문번호']}-{t['조문가지번호']}")#({t['조문시행일자'] if '조문시행일자' in t else ''})")
        else:
            tmp.append(f"{int(curr)}-{t['조문번호']}")
    if t['조문여부'] == '전문':
        curr = t['조문번호']
tmp2 = list()
curr = -1
for t in json_data_en['Law']['JoSection']['Jo']:
    if t['joYn'] == 'Y':
        if 'joBrNo' in t and int(t['joBrNo']) != 0:
            tmp2.append(f"{int(curr)}-{int(t['joNo'])}-{int(t['joBrNo'])}")
        else:
            tmp2.append(f"{int(curr)}-{int(t['joNo'])}")
    if t['joYn'] == 'N':
        curr = t['joNo']

In [ ]:
len(set(tmp2)), len(set(tmp)), len(tmp)

* 법령 : 243117
* 6조 "CDATA[ 제6조(마약류취급자의 허가 등) "에서 en는 개정 이전의 자료도 같이 포함되어 있지만 특별한 구분법이 없음.

In [ ]:
len(set(tmp2)), len(set(tmp)), len(tmp)

In [ ]:
for t, tt in zip(tmp2, tmp):
    print(t, tt)

In [ ]:
len(tmp2), len(tmp)

# 데이터 전처리 (<> 없애기)

In [ ]:
data_kr = []
data_en = []
with open('kr.txt','r') as f:
    data_kr = f.read().split('\n')[:-1]
with open('en.txt','r') as f:
    data_en = f.read().split('\n')[:-1]

In [ ]:
data_en

# 참조만하기

In [ ]:
# 데이터 개수 en: 2586 kr: 5273
# 예상페이지 수: 26page (2586 // 100)
from tqdm import tqdm

same_jo_mst = []
diff_jo_mst = []
err_en_mst = []
err_kr_mst = []

# 페이지에 따라 추출
for page in range(1,27):
    print(page, "/ 26")
    list_url = f'http://www.law.go.kr/DRF/lawSearch.do?OC={OC}&target=elaw&page={page}&display=100'
    response = requests.get(list_url)
    json_data = xmltodict.parse(response.text)

    # 법령별 정보 추출
    for law in tqdm(json_data['LawSearch']['law']):
        mst = law['법령일련번호']
        # en
        url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={mst}'
        response_en = requests.get(url)
        if response_en.status_code != 200:
            err_en_mst.append((mst, response_en.status_code))
            continue

        # kr
        url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={mst}'
        response_kr = requests.get(url)
        if response_kr.status_code != 200:
            err_kr_mst.append((mst, response_kr.status_code))
            continue

        # en, kr parsing
        json_data_en = xmltodict.parse(response_en.text)
        json_data_kr = xmltodict.parse(response_kr.text)

        # check 1
        # 조문개수 확인
        if len(json_data_en['Law']['JoSection']['Jo']) == len(json_data_kr['법령']['조문']['조문단위']):
            same_jo_mst.append(mst)
        else:
            diff_jo_mst.append(mst)


        # print(law['법령상세링크'], law['법령일련번호'])

In [ ]:
data = requests.get('https://www.law.go.kr/lawEngBodyCompareInfoP.do?lsNm=119%EA%B5%AC%EC%A1%B0%E3%86%8D%EA%B5%AC%EA%B8%89%EC%97%90%20%EA%B4%80%ED%95%9C%20%EB%B2%95%EB%A5%A0&lsId=011349&lsiSeq=228097&gubun=EngLs&ancYnChk=')
data.text

In [ ]:
from bs4 import BeautifulSoup
import json

soup = BeautifulSoup(data.text, 'html.parser')
soup.find_all('p')

# 사용할 법령 따로 저장
[기본설정]부분을 실행 후 진행

(error가 발생한 mst는 제외)

In [ ]:
# 857
from tqdm import tqdm

same_jo_mst = []
diff_jo_mst = []
err_mst = []

# 법령별 정보 추출
for mst in tqdm(target_mst):
    try:
        # en
        url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={mst}'
        response_en = requests.get(url)
        if response_en.status_code != 200:
            err_mst.append((mst, response_en.status_code, 'en'))
            continue

        # kr
        url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={mst}'
        response_kr = requests.get(url)
        if response_kr.status_code != 200:
            err_mst.append((mst, response_kr.status_code, 'kr'))
            continue

        # en, kr parsing
        json_data_en = xmltodict.parse(response_en.text)
        json_data_kr = xmltodict.parse(response_kr.text)

        # check 1
        # 조문개수 확인
        if len(json_data_en['Law']['JoSection']['Jo']) == len(json_data_kr['법령']['조문']['조문단위']):
            same_jo_mst.append(mst)
        else:
            diff_jo_mst.append(mst)
    except:
        err_mst.append((mst, 'unknown','need to check'))

In [ ]:
BASE = '/content/drive/MyDrive/취업/일지/230509_국민은행_1차합격/인턴십/AI1차과제/Data/'
lang_type = 'en'
for mst in tqdm(same_jo_mst):
    url = f'http://www.law.go.kr/DRF/lawService.do?target=elaw&OC={OC}&MST={mst}'
    response_en = requests.get(url)
    json_data_en = xmltodict.parse(response_en.text)
    with open(BASE+lang_type+f"/en_{mst}.json", 'w') as f:
        json.dump(json_data_en, indent=4, ensure_ascii=False, fp=f)

In [ ]:
BASE = '/content/drive/MyDrive/취업/일지/230509_국민은행_1차합격/인턴십/AI1차과제/Data/'
lang_type = 'kr'
for mst in tqdm(same_jo_mst):
    url = f'http://www.law.go.kr/DRF/lawService.do?target=law&OC={OC}&MST={mst}'
    response_kr = requests.get(url)
    json_data_kr = xmltodict.parse(response_kr.text)
    with open(BASE+lang_type+f"/kr_{mst}.json", 'w') as f:
        json.dump(json_data_kr, indent=4, ensure_ascii=False, fp=f)

In [ ]:
BASE = ''
lang_type = 'kr'
with open(BASE+lang_type+f"/{'test'}.json", 'w') as f:
    json.dump(json_data_kr, indent=4, ensure_ascii=False, fp=f)

In [ ]:
with open(BASE+lang_type+f"/{'test'}.json", 'r') as f:
    t = json.load(f)